In [1]:
from bc_power import hydro
from bc_power import utils
import yaml
import atlite
import geopandas as gpd
import pandas as pd
from collections import namedtuple

In [3]:
# Script: create_hydro_cutout.py
# Description: main script for creating the hydro cutout based on hydro site locations and the basins they are located within
# and each basins upstream basins.

# Configuration inputs:
config_file = r"/mnt/c/Users/pmcw9/Delta-E/PICS/PyPSA_BC/config/config.yaml"

# main():
# (i) read needed data
cfg = utils.load_config(config_file)
na_basin_data = hydro.load_hydro_basins(cfg["basin_files"]["na_file"])
ar_basin_data = hydro.load_hydro_basins(cfg["basin_files"]["artic_file"])
basin_data = gpd.GeoDataFrame(pd.concat([na_basin_data, ar_basin_data]))
hydro_sites = hydro.load_hydro_sites(cfg["generation"]["hydro_assets"]) # (NEEDS TO BE ADDED)
# hydro_sites = gpd.GeoDataFrame([['Mica', -118.565260, 52.078632, 2971],
#                             ['Revelstoke', -118.192934, 51.050299, 2500]],
#                          columns=['name', 'lon', 'lat', 'capacity']
#                          ).set_index('name')

dx,dy = cfg["cutout"]["dx"], cfg["cutout"]['dy']
time_horizon = slice(cfg["snapshots"]['start'][0],
                     cfg["snapshots"]['end'][0])
# (ii) call function to determine the bounds of the basins
bounds = hydro.get_hydro_cutout_bounds(hydro_sites, basin_data)

# (iii) Create the cutout based on bounds found from above
features = ['height','runoff']
hydro.create_cutout(cfg['cutout']['file'], bounds, dx, dy, time_horizon, features)